In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# one hot encoding "Geography"

onehot_encoder_geo= OneHotEncoder(handle_unknown="ignore")
geo_encoded= onehot_encoder_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_dataframe = pd .DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

geo_encoded_dataframe

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
df= pd.concat([df.drop("Geography",axis=1),geo_encoded_dataframe],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
df.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df["Exited"].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [10]:
df["Geography_France"].value_counts()

Geography_France
1.0    5014
0.0    4986
Name: count, dtype: int64

In [11]:
df["Geography_Germany"].value_counts()

Geography_Germany
0.0    7491
1.0    2509
Name: count, dtype: int64

In [12]:
df["Geography_Spain"].value_counts()

Geography_Spain
0.0    7523
1.0    2477
Name: count, dtype: int64

In [13]:
df["Gender"].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [14]:
X=df.drop(columns=["Exited"])
y=df["Exited"]
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [15]:
X_train.shape

(8000, 12)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
X_train_scaled

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [18]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model = Sequential()

model.add(Dense(3,activation='sigmoid',input_dim=12))
model.add(Dense(1,activation='sigmoid'))

C:\Users\verma\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 3)              │            39 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43 (172.00 B)

 Trainable params: 43 (172.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [22]:
history=model.fit(
    X_train_scaled,y_train
    ,epochs=10
    
    
    )

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.1671
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8534
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6846
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5965
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5539
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5318
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5194
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5072
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4939
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4900


In [23]:
model.layers[0].get_weights()

[array([[ 5.4994654e-02, -1.2813760e-01, -3.4583554e-01],
        [-5.7373017e-01, -6.1926015e-02, -2.4908619e-01],
        [ 1.3357480e+00, -2.1629801e-01,  1.2086871e+00],
        [ 1.5776870e-01, -2.2104786e-01, -2.2067767e-01],
        [ 5.5282271e-01,  3.2585371e-02, -6.1948452e-04],
        [ 2.7991505e-03,  9.7026521e-01, -5.8855492e-01],
        [-2.6819935e-01,  5.1203555e-01, -9.6800262e-03],
        [-3.9944783e-01,  6.1931175e-01, -6.0724556e-01],
        [-1.8104136e-01,  3.2513630e-01,  2.1604294e-01],
        [-7.1928823e-01,  5.0378478e-01, -3.2453293e-01],
        [-2.9904952e-02, -4.7714421e-01,  2.8044537e-01],
        [-5.9189504e-01, -9.0497516e-02, -2.0419063e-01]], dtype=float32),
 array([-0.6446376 , -0.03919962, -0.8212667 ], dtype=float32)]

In [24]:
y_log = model.predict(X_train_scaled)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [25]:
y_pred= np.where(y_log>0.5,1,0)
y_pred = y_pred[:len(y_test)]

In [26]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.8035